In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (15,5)
plt.rcParams['axes.grid'] = False
import seaborn as sns
sns.set_style("whitegrid", {'axes.grid' : False})

In [92]:
from helper_functions_rr import *
from hurst import *
from MCMC.MCMC import MCMC
import random

In [49]:
temp_og, temp_og1 = get_data_BTC()

In [98]:
def return_nmi_bw_mdayH_ndayFReturn(df):
    def alpha(*args):
        m = int(round(args[0]))
        return temp_og[f"H{m}"]
    def prior(params):
        if (params[0] < 100) | (params[0] > 200):
            return 0
        return 1

    iters = 1000
    for i in range(100,201):
        df[f"H{i}"] = df["Close"].rolling(i).apply(lambda x: compute_Hc(x.to_numpy(), kind='price', simplified=False)[0])

    res = []
    for n in range(1,30):
        df["FReturn"] = df["Close"].pct_change(-n)
        temp_og = df.dropna()
        guess = [random.randint(100,200)]
        mc = MCMC(alpha_fn=alpha, alpha_fn_params_0=guess,
                              target=temp_og["FReturn"], num_iters=iters,
                              prior=prior, optimize_fn=None, lower_limit=100, upper_limit=200)
        rs = mc.optimize()

        res_iter = [{"N":n, "M": int(round(mc.analyse_results(rs, top_n=iters)[0][i][0])),
                     "NMI": mc.analyse_results(rs, top_n=iters)[1][i]} \
                      for i in range(iters)]
        res = res + res_iter
    res = pd.DataFrame(res)
    res = res.sort_values("NMI", axis=0, ascending=False).reset_index(drop=True)
    res.drop_duplicates(keep='first', inplace=True, ignore_index=True)
    return res

In [99]:
results = return_nmi_bw_mdayH_ndayFReturn(temp_og)

In [100]:
results


,N,M,NMI
0,24,132,0.591949
1,24,128,0.591341
2,24,129,0.591180
3,24,127,0.590922
4,21,127,0.590784
...,...,...,...
2909,1,177,0.412019
2910,1,176,0.411553
2911,1,174,0.411519
2912,1,178,0.408841
